# **Title**

## **Overview**

## **Data Description**

## **Initial Assumptions and Predictions About the Data**

## **Exploratory Data Analysis**

### **Data Cleaning**

In [283]:
# importing the relevant libraries

# cleaning libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# visualization libraries
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from tabulate import tabulate

# for clustering

# for hypothesis test

In [285]:
# loading in the datasets

# defining names of csv files with similar structures
data_names = ['inflation', 'gdp', 'gdp_growth', 'exports', 'imports',
              'current_account', 'population', 'unemployment', 'fdi']

# for loop to assign variables for each file and creating copy
for name in data_names:
    globals()[name] = pd.read_csv(f"{name}.csv", skiprows=4) 
    globals()[f"{name}_og"] = globals()[name].copy()

# loading hdi file and making copy
hdi = pd.read_csv("hdi.csv")
hdi_og = hdi.copy()

# loading covid file and making copy
covid = pd.read_csv("covid.csv")
covid_og = covid.copy()

In [287]:
for name in data_names:
    indicator = globals()[name]  # Get the DataFrame by its variable name

    # Step 1: Remove the last column
    indicator = indicator.iloc[:, :-1]

    # Step 2: Calculate the mean from column index 4 to all columns except the last four
    mean_pre_covid = indicator.iloc[:, 4:-4].mean(axis=1)

    # Step 3: Calculate the mean of the last four columns and drop rows with >2 nulls in those columns
    post_covid = indicator.iloc[:, -4:]
    mean_post_covid = post_covid.mean(axis=1)
    indicator = indicator[post_covid.isnull().sum(axis=1) <= 2]

    # Optionally, store the computed means in the DataFrame
    indicator[f"{name}_pre_covid"] = mean_pre_covid
    indicator[f"{name}_post_covid"] = mean_post_covid

    # Save back the modified DataFrame
    globals()[name] = indicator

In [288]:
econ_indicators = globals()[data_names[0]][["Country Code", "Country Name", data_names[0] + "_pre_covid", data_names[0] + "_post_covid"]]

for name in data_names[1:]:
    indicator = globals()[name][["Country Code", name + "_pre_covid", name + "_post_covid"]]
    
    # Merge on "Country Code"
    econ_indicators = pd.merge(econ_indicators, indicator, on="Country Code", how="inner")  # Use 'inner' to keep only common countries

econ_indicators

,Country Code,Country Name,inflation_pre_covid,inflation_post_covid,gdp_pre_covid,gdp_post_covid,gdp_growth_pre_covid,gdp_growth_post_covid,exports_pre_covid,exports_post_covid,imports_pre_covid,imports_post_covid,current_account_pre_covid,current_account_post_covid,population_pre_covid,population_post_covid,unemployment_pre_covid,unemployment_post_covid,fdi_pre_covid,fdi_post_covid
0,AGO,Angola,381.436505,20.756307,3.729440e+10,7.605777e+10,3.694286,-0.098247,52.223582,42.827060,37.812202,26.457762,0.096506,7.657086,1.371106e+07,35092124.00,16.578207,15.40700,3.969016,-4.804052
1,ALB,Albania,16.303718,3.786831,6.182332e+09,1.895947e+10,2.945241,3.604788,21.012692,32.383986,39.574366,43.191705,-6.214363,-5.822428,2.708297e+06,2793294.00,16.572724,10.85225,4.212986,7.059969
2,ARM,Armenia,140.826284,4.754400,6.053604e+09,1.752997e+10,3.492637,4.875000,28.263119,43.837123,49.939059,48.733906,-9.637468,-2.363960,2.946195e+06,2970975.00,12.612517,15.06700,4.484251,2.628302
3,AUS,Australia,4.751805,3.975482,4.514917e+11,1.576016e+12,3.440447,2.418989,16.695656,24.603561,17.598784,20.399912,-4.194437,1.642050,1.707379e+07,26002001.75,6.574552,4.74200,2.368805,2.355060
4,AUT,Austria,3.252999,5.127395,1.776161e+11,4.745809e+11,2.759937,0.700001,39.599335,57.260922,38.942242,55.783541,2.331303,1.390939,7.844922e+06,9011568.25,4.893724,5.47900,1.338260,1.198459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,VUT,Vanuatu,4.339635,4.783980,3.838185e+08,1.002087e+09,2.493616,-0.603947,43.232998,13.456422,53.576085,51.727416,-5.492992,-8.922381,1.566333e+05,309545.25,6.907483,4.75925,6.993214,2.623239
141,WSM,Samoa,6.385071,4.175391,3.304428e+08,8.709892e+08,2.196606,-1.727369,29.343690,20.475458,49.316866,53.027829,-4.430853,-6.728470,1.677424e+05,214411.75,5.613345,6.01800,0.870636,0.600371
142,ZAF,South Africa,7.903933,5.233836,1.551726e+11,3.866202e+11,2.951881,0.349020,24.518488,31.201372,22.558080,27.987995,-1.149598,0.914699,3.800774e+07,61913944.50,23.762862,32.14750,0.757135,3.442005
143,ZMB,Zambia,38.223546,14.907891,7.264963e+09,2.424398e+10,3.305469,3.507258,32.255681,44.994100,34.567751,33.244541,-6.831698,6.331545,8.633757e+06,19884976.25,11.970276,5.78300,3.521270,0.806051


In [281]:
match_count = econ_indicators["Country Code"].isin(hdi["ISO3"]).sum()
print(match_count)

144
